# LLM Generation Parameters

## Import Libraries

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [3]:
# Always remember to do this! - What does this do?
load_dotenv(override=True)

True

## Exploring Parameters

### OpenAI Chat Completions API (most widely used)

`OpenAI` Official documentation is here - https://platform.openai.com/docs/api-reference/chat/create Refer to this for a full set of parameters that we'll later explore. 

The most used parameters are as listed below, with definitions from official `OpenAI` API documentation. These are considered "common" because they fundamentally control how an autoregressive language model samples tokens to generate output. 

- **max_tokens** (only *max_completion_tokens* is supported by o1 series; no default): The maximum number of completion tokens that may be used over the course of the run. The run will make a best effort to use only the number of completion tokens specified, across multiple turns of the run. If the run exceeds the number of completion tokens specified, the run will end with status `incomplete`.
- **temperature** (*1 by default*): What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
- **top_p** (*1 by default*): An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
We generally recommend altering this or temperature but not both.
- **top_k**: not available in OpenAI client library
- **frequency_penalty** (*0 by default*): Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.
- **presence_penalty** (*0 by default*): Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
- **stop** (*null by default*): Not supported with latest reasoning models o3 and o4-mini. Up to 4 sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

### Anthropic's Messages API

It's good to note that Anthropic's `Messages` API contains similar parameters except a few differences:

- **max_tokens** is used as is
- **temperature** has a value between 0 and 1
- **top_p** is as is
- **top_k** is also available: Only sample from the top K options for each subsequent token. Used to remove "long tail" low probability responses. Recommended for advanced use cases only. You usually only need to use temperature.
- **frequency_penalty** and **presence_penalty** parameters are not available.
- **stop_sequences** is used instead of **stop**

### Some important considerations

- Exact naming conventions of parameters can vary, but functionality is same (like we saw with Anthropic's API).
- The optimal range or impact of a parameter might differ between models, even if general principle is same. A temperature of 1.0 might be very wild for one model but only moderately creative for another.
- Differences between different parameter configurations will be more prominent in larger LLMs, as with smaller LLMs, the "spread" of probabilities for the next token might not be as wide.

### Setting temperature vs. top_p for different use cases: 

| Use Case               | Temperature | Top_p | Description                                                                                        |
| :--------------------- | :---------- | :---- | :------------------------------------------------------------------------------------------------- |
| Code Generation        | 0.2         | 0.1   | Generates code that adheres to established patterns and conventions. Output is more deterministic and focused. Useful for generating syntactically correct code. |
| Creative Writing       | 0.7         | 0.8   | Generates creative and diverse text for storytelling. Output is more exploratory and less constrained by patterns. |
| Chatbot Responses      | 0.5         | 0.5   | Generates conversational responses that balance coherence and diversity. Output is more natural and engaging. |
| Code Comment Generation | 0.3         | 0.2   | Generates code comments that are more likely to be concise and relevant. Output is more deterministic and adheres to conventions. |
| Data Analysis Scripting | 0.2         | 0.1   | Generates data analysis scripts that are more likely to be correct and efficient. Output is more deterministic and focused. |
| Exploratory Code Writing | 0.6         | 0.7   | Generates code that explores alternative solutions and creative approaches. Output is less constrained by established patterns. |

References from here: https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api/172683/10

### Setting Up

In [4]:
# Use either Llama3.2 (3B) with Ollama
!ollama pull llama3.2    

# Initialize the OpenAI client to connect to Ollama
llm_api = OpenAI(base_url='http://localhost:11434/v1', 
                 api_key='ollama') # note that llm_api is a good variable name, because the LLM is indeed exposed locally as an API by Ollama!

# Define the model to use
model_name = "llama3.2" # Ensure this matches the pulled model in Ollama

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# # Or use DeepSeek-R1-Distill-Llama-70B from Groq
# llm_api = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

# # Define the model to use
# model_name = "deepseek-r1-distill-llama-70b"

In [6]:
#Helper function for text generation
def generate_and_display(prompt, param_name, param_value, num_runs=1, **kwargs):
    """
    Helper function to make a call to the LLM and display results.
    Runs multiple times for stochastic parameters to highlight differences.
    """
    print(f"\n--- Demonstrating {param_name}={param_value} ---")
    print(f"Prompt: \"{prompt}\"")
    print(f"Other consistent parameters: {kwargs}")

    messages = [
        {"role": "system", "content": "You are a helpful and concise AI assistant for a business."},
        {"role": "user", "content": prompt},
    ]

    for i in range(num_runs):
        print(f"\n--- Run {i+1} ---")
        try:
            response = llm_api.chat.completions.create(
                model=model_name,
                messages=messages,
                **{param_name: param_value, **kwargs}
            )
            print("Generated Text:")
            answer = response.choices[0].message.content.strip()
            display(Markdown(answer)) # .strip() removes leading/trailing whitespace for cleaner output
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Ensure Ollama is running and the model is pulled. Skipping further runs for this parameter setting.")
            break # Exit loop if an error occurs

    print("\n--- End Demonstration for this parameter setting ---\n" + "="*80)

### 1. Temperature

**Commercial Use Cases**: Content generation (marketing copy, ad variations, blog post ideas), creative brainstorming for product features, unique customer engagement messages.

**What to expect:**
- **Low Temperature**: Look for very similar, factual, and direct outputs across runs. Less "fluff," more to the point.
    - E.g., Consistent, factual product bullet points for a catalog
- **Medium Temperature**: Outputs will start showing more natural language, some variation in phrasing, but still coherent.
    - E.g., Engaging marketing copy for a landing page
- **High Temperature**: Expect more varied vocabulary, potentially more metaphorical or unusual phrasing, and noticeable differences between runs. It might sometimes deviate slightly from the core prompt if pushed too high.
    - E.g., Brainstorming unique ad slogans or creative concepts for a campaign

In [8]:
common_prompt_temp = """
Merck is heavily investing in Agentic AI and AI Gateways to personalize healthcare for millions of patients. 
Envision a new Agentic AI-powered service, "Merck Vitals," that leverages patient data and medical knowledge.
First, describe the service focusing on its innovative features and how it will enhance daily life for a typical user.
Then, think about the role of AI gateway in enabling a production-ready solution.
Keep the answer less than 70 tokens."""

In [9]:
# Example 1.1: Low Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 0.1,
    num_runs=3, # Run multiple times to show consistency
    max_completion_tokens=70
)


--- Demonstrating temperature=0.1 ---
Prompt: "
Merck is heavily investing in Agentic AI and AI Gateways to personalize healthcare for millions of patients. 
Envision a new Agentic AI-powered service, "Merck Vitals," that leverages patient data and medical knowledge.
First, describe the service focusing on its innovative features and how it will enhance daily life for a typical user.
Then, think about the role of AI gateway in enabling a production-ready solution.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


**Merck Vitals: Personalized Healthcare Service**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Key features include:

* **Personalized Health Profiles**: AI-driven analysis of patient data, medical history, and lifestyle habits.
* **Predictive Insights**: Proactive alerts for potential health risks and preventive measures.
* **Medication Optimization**: AI-suggested medication regimens based on individual needs.

**AI Gateway Enabling Production-Ready Solution**

The AI gateway plays a crucial role in integrating Merck Vitals with various healthcare systems, ensuring seamless data exchange and secure patient data management. The gateway enables:

* **Data Standardization**: Normalizing patient data for analysis and processing.
* **Integration with Wearables & Devices**: Connecting patients' wearable devices to the platform.
* **Secure Data Storage**: Safeguarding sensitive patient information.

By leveraging these innovative features, Merck Vitals enhances daily life for typical users by providing proactive guidance, empowering informed decision-making, and improving overall health outcomes.


--- Run 2 ---
Generated Text:


**Merck Vitals: Personalized Healthcare Service**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Key features include:

* **Personalized Health Profiles**: AI-driven analysis of patient data, medical history, and lifestyle habits creates tailored recommendations for wellness and disease prevention.
* **Predictive Insights**: Advanced analytics identify potential health risks, enabling proactive interventions and early treatment.
* **Virtual Coaching**: AI-powered chatbots offer guidance on healthy habits, medication adherence, and symptom management.

**AI Gateway Enabling Production-Ready Solution**

The AI gateway plays a crucial role in integrating Merck Vitals with various healthcare systems, ensuring seamless data exchange and secure patient data protection. The gateway enables:

* **Data Standardization**: Normalizing patient data for analysis and processing.
* **Integration with Wearables & Devices**: Connecting patients' wearable devices and medical equipment to the platform.
* **Secure Data Transmission**: Ensuring HIPAA compliance and protecting sensitive patient information.

By leveraging AI gateways, Merck Vitals can provide a comprehensive, production-ready solution that revolutionizes personalized healthcare for millions of patients.


--- Run 3 ---
Generated Text:


**Merck Vitals: Personalized Healthcare Service**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Key features include:

* **Personalized Health Profiles**: AI-driven analysis of patient data, medical history, and lifestyle habits creates a tailored profile.
* **Predictive Insights**: AI forecasts potential health risks and recommends preventive measures.
* **Medication Optimization**: AI suggests optimal medication regimens based on individual needs.

**AI Gateway Enabling Production-Ready Solution**

The AI gateway plays a crucial role in integrating patient data from various sources, ensuring seamless communication between devices, and enabling real-time updates. This enables Merck Vitals to provide:

* **Real-time Insights**: Patients receive immediate feedback on their health status.
* **Streamlined Care**: Healthcare providers have access to comprehensive patient profiles for informed decision-making.

By leveraging AI gateways, Merck Vitals can deliver a production-ready solution that revolutionizes personalized healthcare.


--- End Demonstration for this parameter setting ---


In [10]:
# Example 1.2: Medium Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 0.7,
    num_runs=3, # Run multiple times to show controlled variation
    max_completion_tokens=70
)


--- Demonstrating temperature=0.7 ---
Prompt: "
Merck is heavily investing in Agentic AI and AI Gateways to personalize healthcare for millions of patients. 
Envision a new Agentic AI-powered service, "Merck Vitals," that leverages patient data and medical knowledge.
First, describe the service focusing on its innovative features and how it will enhance daily life for a typical user.
Then, think about the role of AI gateway in enabling a production-ready solution.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


**Merck Vitals**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Its innovative features include:

* Personalized health dashboards with real-time insights
* AI-driven risk prediction and prevention recommendations
* Virtual consultations with healthcare professionals
* Integrated medication management and adherence tracking

These features enhance daily life by providing patients with actionable intelligence, increasing confidence in self-care, and improving overall well-being.

**AI Gateway Role**

The AI gateway plays a crucial role in enabling a production-ready solution for Merck Vitals. It connects to various data sources, aggregates patient information, and applies machine learning algorithms to provide actionable insights. The gateway ensures seamless integration of disparate systems, providing a secure and scalable platform for advanced analytics and decision-making.


--- Run 2 ---
Generated Text:


**Merck Vitals: Personalized Healthcare**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Innovative features include:

* **Personalized Medicine Plans**: AI-driven recommendations based on individual medical history, lifestyle, and genetic profiles.
* **Real-time Health Insights**: Continuous monitoring and analysis of vital signs, medication interactions, and predictive analytics for early disease detection.
* **Smart Health Coaching**: AI-assisted guidance for healthy habits, wellness tips, and medication adherence.

**AI Gateway**

The Merck Vitals AI gateway seamlessly integrates patient data from various sources, ensuring a comprehensive view of each individual's health. This gateway enables:

* Secure data exchange between healthcare providers and patients.
* Automated data standardization and analytics.
* Advanced security measures to protect sensitive medical information.

By harnessing the power of Agentic AI and AI gateways, Merck Vitals revolutionizes patient engagement and personalized care, enhancing daily life and improving health outcomes.


--- Run 3 ---
Generated Text:


**Merck Vitals: Personalized Healthcare**

Merck Vitals is an Agentic AI-powered service that empowers patients to take control of their health. Key features include:

* **Personalized Health Insights**: AI-driven analysis of patient data, medical history, and lifestyle habits provides actionable recommendations for optimal well-being.
* **Predictive Analytics**: Advanced algorithms identify potential health risks, enabling early interventions and preventive care.
* **Virtual Care Coaching**: AI-powered chatbots offer personalized guidance on healthy behaviors, medication adherence, and disease management.

**AI Gateway Role**

The AI gateway plays a crucial role in enabling Merck Vitals by:

* Integrating with various electronic health records (EHRs) and wearables
* Providing real-time data processing and analytics capabilities
* Ensuring seamless communication between patients, healthcare providers, and the AI system


--- End Demonstration for this parameter setting ---


In [11]:
# Example 1.3: High Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 1.5, # Pushing temperature higher for more dramatic effect
    num_runs=3, # Run multiple times to show increased randomness
    max_completion_tokens=70
)


--- Demonstrating temperature=1.5 ---
Prompt: "
Merck is heavily investing in Agentic AI and AI Gateways to personalize healthcare for millions of patients. 
Envision a new Agentic AI-powered service, "Merck Vitals," that leverages patient data and medical knowledge.
First, describe the service focusing on its innovative features and how it will enhance daily life for a typical user.
Then, think about the role of AI gateway in enabling a production-ready solution.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


Introducing Merck Vitals, an Agentic AI-powered service that harmonizes personalization with healthcare. This innovative service features:

* Data-driven insights to foster meaningful patient-provider conversations
* Real-time monitoring of vital signs and medication management
* Personalized treatment plans tailored to each patient's unique needs
* AI-gated self-care tools for seamless life coordination

To bring this vision to production, Merck implements AI gateways that manage data privacy, security, and interoperability. By integrating Agentic AI with the gateway's robust architecture, patients can enjoy an intuitive, user-friendly experience while ensuring rigorous cybersecurity and regulatory compliance worldwide


--- Run 2 ---
Generated Text:


**Merck Vitals: Empowering Personalized Healthcare**

Merck Vitals is an Agentic AI-powered service that combines patient data and medical expertise to provide tailored health insights and recommendations. Innovative features:

* **Proactive health tracking**: AI-analyzes user data (wearables, sensors, medical records) to identify early warning signs of potential illnesses.
* **Personalized health plans**: Offers AI-generated strategies to mitigate or reverse adverse conditions, incorporating genomic information and lifestyle factors.
* **Enhanced medication efficacy**: Intelligent refashioning of treatments based on a patient's response history.

AI Gateway acts as an enabler by:

* Securing seamless clinical data integration
* Processing large datasets simultaneously for efficient analysis.
* Validating AI-derived suggestions through peer-reviewed medical sources, ensuring quality assurance.


--- Run 3 ---
Generated Text:


**Merck Vitals:** A Personalized Health Service

Merck Vitals is an agentic AI-powered service that harnesses patient data and medical knowledge to Provide tailored wellness recommendations, proactive alerts for preventable conditions, and insightful analytics for health equity assessment.

**Innovative Features:**

1. **Patient-centric risk assessment:** AI predicts individual health risks, prompting targeted interventions.
2. **Symptom monitoring ecosystem:** Track symptoms through voice chatbots or wearables, accessing expert-approved analysis.
3. **Peer support networks:** Engage in secure communities based on diagnoses and interests for mutual support.

**AI Gateway Integration:**
An optimized AI gateway facilitates seamless flow of integrated clinical, genomic, and lifestyle data to foster more comprehensive predictive models enabling personalized health journeys


--- End Demonstration for this parameter setting ---


### 2. Max_tokens

**Commercial Use Cases**: Chatbot response length control, email subject line generation, tweet generation, summarizing lengthy documents for internal reports, generating specific-length ad copy.

**What to expect**: 

- The generated output will strictly adhere to the `max_tokens` limit, regardless of whether the summary is complete.
- Depending on the use case, the `max_tokens` parameter can be set:
    - Very few tokens (e.g., for a headline or quick alert)
    - Moderate tokens (e.g., for an executive summary or internal Slack message)
    - More tokens (e.g., for a detailed summary section in a newsletter or report)

In [12]:
common_prompt_max_tokens = """
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."""

In [13]:
# Example 2.1: Very few tokens
generate_and_display(
    common_prompt_max_tokens + " Complete response in less than 15 tokens.",
    "max_tokens", 15,
    temperature=0.5
)


--- Demonstrating max_tokens=15 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles. Complete response in less than 15 tokens."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist, I'd leverage Generative AI to:

1.


--- End Demonstration for this parameter setting ---


In [14]:
# Example 2.2: Moderate tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 50,
    temperature=0.5
)


--- Demonstrating max_tokens=50 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd love to explore the potential of Generative AI (GA) in creating novel training drills and strategic simulations for batsmen. Here's how GA can be leveraged:

**Data Collection**

To


--- End Demonstration for this parameter setting ---


In [15]:
# Example 2.3: More tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 150,
    temperature=0.5
)


--- Demonstrating max_tokens=150 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd love to explore the potential of Generative AI (GAI) in creating novel training drills and strategic simulations for batsmen. Here's how:

**Data Collection and Analysis**

To leverage GAI effectively, we'll need access to large datasets containing information on:

1. Batting performances and statistics
2. Bowling styles, including types (fast, spin, etc.), deliveries (short-pitched, yorkers, etc.), and variations (leg-spin, off-spin, etc.)
3. Pitch conditions, including surface type (dry, damp, etc.), weather conditions (temperature, humidity, etc.), and ball tracking data
4. Historical matches and key events (dismissals, boundaries,


--- End Demonstration for this parameter setting ---


In [16]:
#Example 2.4: Even more max_tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 1000,
    temperature=0.5
)


--- Demonstrating max_tokens=1000 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd love to explore the potential of Generative AI in creating novel training drills and strategic simulations for batsmen. Here's how:

**Training Drills:**

1. **Adaptive Batting Drills:** Utilize Generative AI to create customized batting drills based on individual batsmen's strengths, weaknesses, and playing styles. The AI can analyze data from past matches, identify patterns, and generate new drill scenarios that cater to the batsman's needs.
2. **Pitch-Specific Training:** Leverage Generative AI to design training drills tailored to specific Indian pitches (e.g., Eden Gardens, Wankhede Stadium, or Feroz Shah Kotla). The AI can analyze data on ball movement, bounce, and spin patterns for each pitch to create drills that mimic the unique challenges of those surfaces.
3. **Batting Against Different Bowlers:** Use Generative AI to generate training scenarios where batsmen face various bowling styles (e.g., fast bowler, spinner, or all-rounder). The AI can analyze data on bowlers' strengths, weaknesses, and tactics to create drills that challenge the batsman in different ways.

**Strategic Simulations:**

1. **Batting Lineup Optimization:** Employ Generative AI to optimize batting lineups based on team performance, opponent analysis, and player availability. The AI can generate simulations of different batting combinations, taking into account factors like wicket-keeping, fielding positions, and batting order.
2. **Pitch Selection and Strategy:** Utilize Generative AI to analyze data on pitch conditions, opposition strengths, and weather forecasts to predict optimal pitch selection for the team. The AI can also simulate different batting strategies (e.g., playing aggressively or defensively) based on the expected conditions.
3. **Adaptive Batting Plans:** Leverage Generative AI to create adaptive batting plans that adjust to changing game situations. For example, if the opposition is losing wickets, the AI can generate a plan that focuses on scoring quickly and taking advantage of their vulnerability.

**Benefits:**

1. **Increased Efficiency:** Generative AI can significantly reduce the time spent on traditional training methods, allowing the team to focus on more high-intensity, results-driven sessions.
2. **Improved Accuracy:** By analyzing vast amounts of data, Generative AI can identify patterns and trends that human analysts might miss, providing a more accurate assessment of batsmen's strengths and weaknesses.
3. **Enhanced Player Development:** The personalized training drills and strategic simulations generated by Generative AI can help players develop their skills at an accelerated pace, leading to improved overall performance.

**Technical Requirements:**

1. **Data Collection:** Access to large datasets on batting performances, bowling styles, pitch conditions, and weather forecasts.
2. **AI Algorithm:** Adoption of a suitable Generative AI algorithm (e.g., GANs or Variational Autoencoders) that can process and analyze the collected data effectively.
3. **Cloud Computing:** Utilization of cloud computing resources to support the processing and simulation requirements of Generative AI.

By leveraging Generative AI in this way, the IPL team can unlock new levels of innovation and effectiveness in batting training and strategic planning, ultimately giving them a competitive edge in the high-stakes world of Indian Premier League cricket.


--- End Demonstration for this parameter setting ---


### 3. Top_p (Nucleus sampling)

**Commercial Use Cases**: Generating diverse customer service responses (without going off-topic), creating varied product names, generating slightly different content variations for A/B testing.

**What to Observe**:

- **Low** top_p: Outputs will be very similar across runs, often picking the most statistically common words/phrases. Less variation in phrasing.

- **Medium** top_p: Some variations in phrasing will emerge, but still within a coherent and expected range.

- **High** top_p: More diverse and potentially more "creative" word choices and phrasing, showing greater variation across runs.

In [17]:
common_prompt_top_p = """
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."""

In [18]:
# Example 3.1: Low top_p (for a very standard, consistent customer service greeting)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.1,
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.1 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in trying our new premium coffee blend. I'd be happy to tell you more about it.

Our expertly crafted blend is indeed a game-changer, and we're confident it will rival the best of the big names like Starbucks, Peet's Coffee, and Cafe Coffee Day. What sets us apart, however, is


--- Run 2 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in trying our new premium coffee blend. I'd be happy to tell you more about it.

Our expertly crafted blend is indeed a game-changer, and we're confident it will rival the best of the big names like Starbucks, Peet's Coffee, and Cafe Coffee Day. What sets us apart, however, is


--- Run 3 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in trying our new premium coffee blend. I'd be happy to tell you more about it.

Our expertly crafted blend is indeed a game-changer, and we're confident it will rival the best of the big names like Starbucks, Peet's Coffee, and Cafe Coffee Day. What sets us apart, however, is


--- End Demonstration for this parameter setting ---


In [19]:
# Example 3.2: Medium top_p (for a friendly, slightly varied customer service interaction)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.5,
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.5 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in trying our new premium coffee blend. Our expert roasters have crafted a truly exceptional cup that's sure to satisfy even the most discerning palates.

Our signature blend, 'Elevate,' is carefully selected from the finest Arabica beans sourced from around the world. What sets it apart is its unique flavor profile,


--- Run 2 ---
Generated Text:


"Thank you for stopping by! We're thrilled to introduce our newest addition - a premium coffee blend that's sure to tantalize your taste buds. Our expertly crafted blend is designed to rival the best, and we think it just might give Starbucks, Peets Coffee, and Cafe Coffee Day a run for their money.

This exceptional blend boasts a rich, smooth flavor profile with notes of dark chocolate


--- Run 3 ---
Generated Text:


Here's a warm and inviting response to the customer's inquiry:

"Thank you for your interest in our new premium coffee blend! We're thrilled to introduce you to our latest creation, carefully crafted to rival the best of the industry - Starbucks, Peets Coffee, and Cafe Coffee Day.

Our unique blend is a masterful fusion of expertly sourced Arabica beans from various regions around the world.


--- End Demonstration for this parameter setting ---


In [20]:
# Example 3.3: High top_p (for brainstorming diverse opening lines for marketing outreach)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.99, # Pushing higher for more noticeable effect
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.99 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


Welcome to our shop! We're thrilled you're interested in trying our newest premium coffee blend, which has been making waves among coffee connoisseurs. Our expertly crafted roast is indeed comparable to some of the world's top brands like Starbucks, Peets Coffee, and Cafe Coffee Day.

This unique blend is carefully curated from a selection of rare and exotic beans sourced from small-batch farms around


--- Run 2 ---
Generated Text:


Welcome to [Your Business Name]! We're thrilled to have you try our new premium coffee blend, carefully crafted to rival the best of the industry. Our expert roasters have worked tirelessly to create a truly exceptional cup, with a rich and smooth flavor profile that's sure to delight both coffee connoisseurs and newcomers alike.

Our signature blend is made from a selection of rare and exotic beans


--- Run 3 ---
Generated Text:


We're thrilled to introduce you to our newest premium coffee blend - a game-changer for coffee connoisseurs like yourself! Our expert roasters have carefully crafted this unique blend, inspired by the finest Arabica beans from around the world.

Our premium coffee blend is designed to rival the best of the industry's biggest names: Starbucks, Peets Coffee, and Cafe Coffee Day. With its rich


--- End Demonstration for this parameter setting ---


### 4. Frequency_penalty

**Commercial Use Cases**: Ensuring variety in marketing emails, preventing chatbots from repeating FAQs, generating diverse social media posts about the same product.

**What to Observe**:

- **No** penalty (0.0): Look for words or short phrases being repeated frequently within the generated paragraph.

- **Moderate** penalty: Repetition should be noticeably reduced, leading to more varied sentences.

- **High** penalty: The model will actively avoid repeating words, potentially leading to more complex or less natural phrasing if it has to find many synonyms.

In [21]:
common_prompt_freq_penalty = """
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"""

In [22]:
# Example 4.1: Negative frequency penalty (encourages repetition)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", -1,
    temperature=0.7,
    seed = 42
)


--- Demonstrating frequency_penalty=-1 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Can you please provide more details?"
3. "I apologize for the inconvenience."
4. "Let me check on that for you."
5. "Our policy is as follows..."
6. "I understand your concern, and I'm here to help."
7. "I'll do my best to resolve this issue."
8. "Can you please confirm your order number?"
9. "I'll escalate this to my supervisor."
10. "I apologize for the delay."
11. "I'd like to offer a solution."
12. "I'll need to verify some information."
13. "I'll provide you with an update."
14. "Is there anything else I can assist you with?"
15. "I'll need to authenticate your account."


--- End Demonstration for this parameter setting ---


In [23]:
# Example 4.2: Moderate frequency penalty (reduced repetition)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", 1.0, # Increased for more impact
    temperature=0.7,
    seed=42
)


--- Demonstrating frequency_penalty=1.0 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Can you please provide more details?"
3. "Let me see what I can do."
4. "Our policy is..."
5. "I apologize for the inconvenience."
6. "Would you like me to escalate this issue?"
7. "That's not possible at this time."
8. "May I ask a few questions?"
9. "Is there anything else I can help with today?"
10. "Can you confirm your order number please?"
11. "I'll check on that for you immediately."
12. "Our current status is..."
13. "What was the issue again?"
14. "Would you prefer to resolve this now or at a later time?"
15. "Is there anything else I can do right now?"


--- End Demonstration for this parameter setting ---


In [24]:
# Example 4.3: High frequency penalty (strong reduction in repetition, might affect coherence)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", 2.0, # Max penalty for strong effect
    temperature=0.7,
    seed=42
)


--- Demonstrating frequency_penalty=2.0 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Can I provide an alternative solution?"
3. "Let me look into this for you."
4."Our policy is as follows..."
5."Would you like me to escalate this issue?"
6.'"We apologize, but [specific reason]."
7."'s the status of your order?
8.""How did we do today? Is there anything else I can help with?"
9. "I'm going to need some more information from you."
10. "Our team will get back to you within 24 hours."
11."Is this an issue or a question?"
12."'s the next step in resolving your concern?
13.""We're here for our valued customers..."
14.""How can I make it right for you today?"
15. ""Let me check on that availability"


--- End Demonstration for this parameter setting ---


### 5. Presence_penalty

**Commercial Use Cases**: Encouraging a customer service bot to explore different solutions/topics, generating comprehensive meeting minutes by covering all points, producing diverse content ideas for a campaign.

**What to Observe**:

- **No** penalty (0.0): The model might dwell on a few initial topics or points without moving on to others mentioned in the conceptual prompt.

- **Moderate** penalty: The summary should cover a broader range of distinct action items or concepts, rather than just elaborating on the first few.

- **High** penalty: The model will try hard to introduce new concepts or distinct ideas, possibly jumping between points quickly or even generating somewhat disjointed output if forced too much.

In [25]:
common_prompt_pres_penalty = """
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"""

In [26]:
# Example 5.1: No presence penalty (might focus heavily on one or two main themes)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 0.0,
    temperature=0.7,
)


--- Demonstrating presence_penalty=0.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Good morning team,

As we explore the potential of Generative AI to enhance our sales and customer engagement processes, I'd like to highlight four key areas where this technology can make a significant impact. Here are some examples:

**1. Personalized Content Creation**

GenAI can help us generate personalized content, such as product descriptions, customer testimonials, or even entire marketing campaigns. For instance, we could use GenAI to create customized email templates that address individual customers' interests and needs.

Example: A customer's name is inserted into a template, and the AI generates a unique email with relevant product recommendations based on their purchase history.

**2. Automated Sales Conversations**

GenAI-powered chatbots can engage customers in real-time, providing personalized support and answering common questions. This can help reduce response times and improve overall customer satisfaction.

Example: A customer asks about a product's features, and the GenAI-powered chatbot responds with relevant information, highlighting key benefits and answering questions.

**3. Data-Driven Lead Scoring**

GenAI can analyze vast amounts of customer data to predict lead behavior and score them accordingly. This allows us to prioritize leads and focus on high-potential customers.

Example: A GenAI algorithm analyzes a large dataset of customer interactions, identifying patterns and predicting the likelihood of conversion for each lead.

**4. Dynamic Content Generation**

GenAI can create dynamic content that adapts to changing market conditions or customer preferences. This enables us to respond quickly to emerging trends and offer highly relevant offers.

Example: A GenAI-powered system analyzes real-time data on a popular competitor's pricing strategy and generates a counteroffer with tailored discounts for our customers, increasing the likelihood of conversion.

These are just a few examples of how Generative AI can enhance our sales and customer engagement processes. Let's discuss these ideas further and explore ways to integrate them into our operations.


--- End Demonstration for this parameter setting ---


In [27]:
# Example 5.2: Moderate presence penalty (encourages covering a broader range of distinct action items)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 1.0, # Increased for more impact
    temperature=0.7,
)


--- Demonstrating presence_penalty=1.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Good morning team,

As we explore the possibilities of Generative AI in our business, I'd like to highlight four key areas where this technology can make a significant impact:

**Area 1: Personalized Sales Content Generation**

With GenAI, we can generate customized sales content, such as product descriptions, social media posts, and marketing emails. For example, an AI-powered tool could analyze customer data and preferences to create personalized product recommendations for each individual.

* Example: A customer has browsed multiple products on our website, but hasn't made a purchase yet. Our GenAI system generates a targeted email with recommended products tailored to their interests.
* Potential impact: Increased sales conversion rates by up to 25%

**Area 2: Chatbot-Powered Customer Engagement**

GenAI can help create more effective chatbots that provide personalized customer support and responses. This enables us to offer 24/7 engagement, reduce response times, and improve overall customer satisfaction.

* Example: A customer contacts our chatbot with a query about their order status. The GenAI system responds promptly with accurate information and helpful next steps.
* Potential impact: Reduced customer care costs by up to 30% through reduced support requests

**Area 3: Dynamic Content Generation for Marketing Campaigns**

GenAI can help generate dynamic content, such as images, videos, and social media posts, that adapt to changing market conditions and audience interests. This enables us to create more engaging marketing campaigns that resonate with our target audience.

* Example: During a product launch, our GenAI system generates eye-catching visuals showcasing the new product features.
* Potential impact: Increased brand visibility by up to 40% through improved engagement metrics

**Area 4: Predictive Lead Scoring and Qualification**

GenAI can help analyze customer data to predict lead behavior, identify potential issues, and qualify leads more accurately. This enables us to focus on high-quality leads that are more likely to convert into sales.

* Example: An AI-powered system analyzes a new lead's browsing history, purchase intent, and demographic information to determine their likelihood of converting.
* Potential impact: Reduced lead-to-opportunity conversion rates by up to 20% through targeted resource allocation

These examples illustrate just a few possible ways GenAI can enhance our sales and customer engagement processes. I believe that with careful implementation, we can unlock significant value from this technology.

Next steps?

Would anyone like to discuss potential implementation strategies or challenges in more detail?


--- End Demonstration for this parameter setting ---


In [28]:
# Example 5.3: High presence penalty (strong encouragement for new ideas, potentially disjointed)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 2.0, # Max penalty for strong effect
    temperature=0.7,
)


--- Demonstrating presence_penalty=2.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Thank you everyone! Today, we'll explore the vast potential of Generative AI to enhance our sales and customer engagement processes.

Here are four key areas where GenAI can make significant improvements:

**1. Personalized Sales Content Generation**
GenAI-powered tools can analyze market trends, customer data, and industry insights to create tailored sales content at scale. For example:
	* Our marketing team uses a GenAI platform to generate product descriptions that highlight unique selling points for each new device release.
	* The AI-generated copy improves conversion rates by 15% compared to traditional manual writing.

**2. Chatbot-Powered Customer Support**
GenAIs can drive more efficient and empathetic customer support experiences through chatbots, reducing response times and improving resolution rates. For instance:
	* Our company deploys a GenAI-powered chatbot that helps customers with basic queries, freeing up human support agents to focus on complex issues.
	* The chatbot's accuracy improves by 22% compared to traditional scripted responses.

**3. Predictive Lead Scoring**
GenAIs can analyze large datasets to identify high-potential leads and predict sales success rates more accurately than manual evaluation methods. For example:
	* We integrate a GenAI-powered lead scoring system that evaluates customer behavior, preferences, and purchase history.
	* The AI scores lead quality with an accuracy of 92%, enabling our sales teams to prioritize their efforts effectively.

**4. Content Recommendation Engine**
GenAIs can use natural language processing (NLP) to analyze vast amounts of content data, providing personalized recommendations for customers based on their interests and preferences. For instance:
	* Our company develops a GenAI-powered content recommendation engine that suggests relevant products or services to our users.
	* The user engagement increases by 25% after implementing the AI-driven recommendation system.

These examples demonstrate how Generative AI can revolutionize various aspects of our sales and customer engagement processes, driving efficiency, accuracy, and improved outcomes.


--- End Demonstration for this parameter setting ---


### 6. Stop

**Commercial Use Cases**: Ensuring chatbot responses don't continue past a natural break, extracting specific data blocks (e.g., JSON), generating bulleted lists that don't overextend, completing fill-in-the-blank forms.
garding:"

**What to Observe**: The model's generation will immediately halt once any of the provided stop sequences appear in its output.

In [29]:
# Example 6.1: Stopping at newline (for single-line completions)
common_prompt_stop = """
Draft a quick customer service response template:\nThank you for contacting TrueFoundry. We received your inquiry regarding:"""
generate_and_display(
    common_prompt_stop,
    "stop", ["\n"],
    temperature=0.5
)


--- Demonstrating stop=['\n'] ---
Prompt: "
Draft a quick customer service response template:
Thank you for contacting TrueFoundry. We received your inquiry regarding:"
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here's a quick customer service response template:


--- End Demonstration for this parameter setting ---


In [34]:
# Example 6.2: Stopping at a specific closing phrase (for structured email generation)
common_prompt_email = """Draft a professional email to a client regarding their contract status. 
Subject: Your Contract #12345 Update\nDear [Client Name],\n\nYour recent contract #12345 has been:"""
generate_and_display(
    common_prompt_email,
    "stop", ["Best regards,"], # Model will stop before adding the closing
    temperature=0.5
)


--- Demonstrating stop=['Best regards,'] ---
Prompt: "Draft a professional email to a client regarding their contract status. 
Subject: Your Contract #12345 Update
Dear [Client Name],

Your recent contract #12345 has been:"
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here's a draft of the email:

Subject: Your Contract #12345 Update

Dear [Client Name],

I am writing to provide you with an update on the status of your contract, #12345. As we previously discussed, our team is actively working to complete the necessary tasks and ensure that all requirements are met.

As of today, I am pleased to inform you that:

[Insert one of the following options]

* The contract has been finalized and is ready for signature.
* There have been some minor revisions required, which will be communicated to you separately.
* Additional information or clarification is needed from your end before we can proceed with the next steps.

Please let me know if you have any questions or concerns regarding this update. I am more than happy to provide further details and keep you informed of our progress.

If you would like to review the contract, please find it attached to this email for your convenience. If you require any assistance or need to schedule a call to discuss, please don't hesitate to reach out.

Thank you for your patience and cooperation. I look forward to hearing from you soon.


--- End Demonstration for this parameter setting ---


In [35]:
# Example 6.3: Stopping after a list item (for controlled list generation)
common_prompt_list = "List 3 benefits of our AI gateway:\n1. AI Governance\n2. Efficient budgeting\n3."
generate_and_display(
    common_prompt_list,
    "stop", ["4."], # Stop before generating the 4th item (assuming it would start with 4.)
    temperature=0.5
)


--- Demonstrating stop=['4.'] ---
Prompt: "List 3 benefits of our AI gateway:
1. AI Governance
2. Efficient budgeting
3."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here is the completed list with the third benefit:

3. Enhanced decision-making: Our AI gateway provides real-time insights and analytics, enabling data-driven decisions that can drive business growth and optimization.

So, to recap, the three benefits of our AI gateway are:

1. **AI Governance**: Ensures the responsible development, deployment, and use of AI within the organization.
2. **Efficient budgeting**: Automates budget allocation and tracking, reducing errors and improving financial management.
3. **Enhanced decision-making**: Provides real-time insights and analytics to inform business decisions and drive growth.


--- End Demonstration for this parameter setting ---


### 7. Seed

**Commercial Use Cases**: Reproducible content for A/B testing (ensuring differences are due to prompt, not randomness), consistent chatbot behavior for quality assurance, debugging prompt engineering, generating consistent test data.

**What to Observe**:

- **Same** seed: The outputs for the multiple runs should be identical or extremely close.
- **Different** seed: The outputs, while still relevant to the prompt, should be distinct from each other across different seeds.

In [36]:
common_prompt_seed = "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."

In [37]:
# Example 7.1: Same seed, same prompt, same parameters (for reproducible slogan generation for testing)
print("\n--- Testing reproducibility with the SAME seed (42) ---")
generate_and_display(
    common_prompt_seed,
    "seed", 42,
    num_runs=3, # Run multiple times to show consistency
    temperature=0.7,
    max_tokens=20
)


--- Testing reproducibility with the SAME seed (42) ---

--- Demonstrating seed=42 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- Run 2 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- Run 3 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- End Demonstration for this parameter setting ---


In [38]:
# Example 7.2: Different seeds, same prompt, same parameters (to get diverse options for review)
print("\n--- Testing different seeds for DIVERSE slogan options ---")
generate_and_display(
    common_prompt_seed,
    "seed", 101,
    num_runs=1, # One run for each distinct seed is enough to show variety
    temperature=0.7,
    max_tokens=20
)


--- Testing different seeds for DIVERSE slogan options ---

--- Demonstrating seed=101 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Sparkle with a Clear Conscience, Clean with Green."


--- End Demonstration for this parameter setting ---


In [39]:
# Example 7.3: Different seeds, same prompt, same parameters (to get diverse options for review)
generate_and_display(
    common_prompt_seed,
    "seed", 202,
    num_runs=1,
    temperature=0.7,
    max_tokens=20
)


--- Demonstrating seed=202 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Clean with a Clear Conscience, Inside and Out."


--- End Demonstration for this parameter setting ---


In [40]:
# Example 7.4: Different seeds, same prompt, same parameters (to get diverse options for review)
generate_and_display(
    common_prompt_seed,
    "seed", 303,
    num_runs=1,
    temperature=0.7,
    max_tokens=20
)


--- Demonstrating seed=303 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- End Demonstration for this parameter setting ---
